In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pickle
import time
import datetime
from pathlib import Path
from tqdm import tqdm
from shapely.ops import Point, LineString
import matplotlib.pyplot as plt

from importlib import reload

In [4]:
traces_fp = Path.home() / 'Documents/BikewaySimData/Projects/gdot/gps_traces'

Distribution of Trips Over Time

In [ ]:
to_plot = trips.copy()

to_plot['year_month'] = to_plot['initial_start_time'].dt.to_period('M')
monthly_counts = to_plot.groupby('year_month').size().reset_index(name='count')
monthly_counts
fig, ax = plt.subplots(figsize=(10, 6))
ax.bar(monthly_counts['year_month'].astype(str),monthly_counts['count'],color='skyblue')
ax.set_title('Number of CycleAtlanta Trips Over Time (Aggregated by Month and Year)', fontsize=16)
ax.set_xlabel('Year-Month', fontsize=14)
ax.set_ylabel('Number of Trips', fontsize=14)
ax.tick_params(axis='both', which='major', labelsize=6)  # Set font size of tick labels
ax.grid(axis='y', linestyle='--', alpha=0.7)
plt.xticks(rotation=45,ha='right')
# Show the plot
plt.tight_layout()
plt.show()

In [10]:
with (traces_fp/'trips_1.pkl').open('rb') as fh:
    full_trips_df = pickle.load(fh)

In [11]:
full_trips_df.columns

Index(['tripid', 'initial_start_time', 'initial_end_time', 'initial_start_lon',
       'initial_start_lat', 'initial_end_lon', 'initial_end_lat',
       'initial_start_X', 'initial_start_Y', 'initial_end_X', 'initial_end_Y',
       'initial_duration', 'initial_total_points', 'initial_avg_accuracy',
       'userid', 'trip_type', 'description', 'start_time', 'end_time',
       'start_lon', 'start_lat', 'end_lon', 'end_lat', 'start_X', 'start_Y',
       'end_X', 'end_Y', 'duration', 'total_points', 'avg_accuracy',
       'max_delta_time', 'mean_delta_time', 'max_distance_ft',
       'avg_distance_ft', 'total_distance_ft', 'first_to_last_ft',
       'max_speed_mph', 'min_speed_mph', 'avg_speed_mph', 'remapped_userid'],
      dtype='object')

In [12]:
cols_needed = ['start_time']


In [17]:
with (traces_fp/'users_1.pkl').open('rb') as fh:
    users = pickle.load(fh)

with (traces_fp/'trips_3.pkl').open('rb') as fh:
    trips_df = pickle.load(fh)

trips_df.reset_index(inplace=True,drop=True)

In [18]:
trips_df

,tripid,userid,remapped_userid,start_X,start_Y,end_X,end_Y,start_label,end_label,sort_start_label,sort_end_label,reverse_trajectory,trip_patterns,trip_pattern_prevalence,trip_type,description,total_distance_ft,avg_speed_mph
0,68,14,14.0,2.238127e+06,1.377715e+06,2.229160e+06,1.373953e+06,0.0,1.0,0.0,1.0,False,0.0,4.0,Commute,Kari,14590.562696,6.336580
1,71,14,14.0,2.226210e+06,1.373792e+06,2.238098e+06,1.377739e+06,2.0,0.0,0.0,2.0,True,0.0,3.0,Commute,Kari,17723.507613,6.940916
2,4865,14,14.0,2.226262e+06,1.373812e+06,2.240540e+06,1.381260e+06,2.0,3.0,2.0,3.0,False,0.0,1.0,Social,Need a northeast exit from piedmont park,27431.269990,6.284932
3,80,18,18.0,2.229754e+06,1.367654e+06,2.229468e+06,1.373925e+06,0.0,1.0,0.0,1.0,False,0.0,1.0,Social,NaN,7716.775258,5.016535
4,110,18,18.0,2.229613e+06,1.373874e+06,2.229545e+06,1.390637e+06,1.0,3.0,1.0,3.0,False,0.0,1.0,Commute,NaN,20068.196577,8.009195
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2733,34187,1723,1723.0,2.234469e+06,1.374973e+06,2.239443e+06,1.359296e+06,-1.0,1.0,-1.0,1.0,False,NaN,NaN,Work-Related,"Grady HS to Home, via BL w/ Rebecca Serna!",30080.838760,9.254977
2734,34242,1733,1733.0,2.230321e+06,1.373114e+06,2.233756e+06,1.361323e+06,0.0,-1.0,-1.0,0.0,True,NaN,NaN,Social,NaN,18289.733133,9.167505
2735,34289,1733,1733.0,2.245986e+06,1.369955e+06,2.257314e+06,1.373259e+06,1.0,-1.0,-1.0,1.0,True,NaN,NaN,Social,NaN,14594.897768,10.451568
2736,34290,1733,1733.0,2.257029e+06,1.371364e+06,2.229939e+06,1.374097e+06,-1.0,0.0,-1.0,0.0,False,NaN,NaN,Social,NaN,35551.538594,12.431327


In [22]:
#add other data
trips_df = trips_df.merge(full_trips_df[['tripid']+cols_needed],on='tripid')

In [28]:
print("Goes from",trips_df['start_time'].min(),"to",trips_df['start_time'].max())

Goes from 2012-10-11 08:53:24 to 2016-09-02 17:27:45


In [35]:
trips_df[trips_df['start_time'].dt.year >= 2016]

,tripid,userid,remapped_userid,start_X,start_Y,end_X,end_Y,start_label,end_label,sort_start_label,sort_end_label,reverse_trajectory,trip_patterns,trip_pattern_prevalence,trip_type,description,total_distance_ft,avg_speed_mph,start_time
957,31984,931,931.0,2.243726e+06,1.369936e+06,2.248279e+06,1.381566e+06,0.0,3.0,0.0,3.0,False,21.0,2.0,Commute,NaN,18576.155879,8.256218,2016-03-01 07:55:39
979,32150,931,931.0,2.238789e+06,1.372410e+06,2.237825e+06,1.414442e+06,0.0,72.0,0.0,72.0,False,0.0,1.0,Commute,NaN,48783.950080,13.159527,2016-03-16 05:06:15
995,33614,931,931.0,2.252714e+06,1.374213e+06,2.248778e+06,1.381516e+06,74.0,3.0,3.0,74.0,True,0.0,1.0,Commute,NaN,9579.537068,5.540473,2016-07-13 07:43:07
1002,31810,931,931.0,2.234466e+06,1.406243e+06,2.257068e+06,1.416734e+06,8.0,70.0,8.0,70.0,False,0.0,1.0,Commute,NaN,33013.222552,14.164496,2016-02-08 16:33:13
1005,33286,931,931.0,2.232732e+06,1.355648e+06,2.239842e+06,1.380878e+06,63.0,11.0,11.0,63.0,True,0.0,1.0,Social,NaN,52749.314704,9.995126,2016-06-17 20:08:42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2733,34187,1723,1723.0,2.234469e+06,1.374973e+06,2.239443e+06,1.359296e+06,-1.0,1.0,-1.0,1.0,False,NaN,NaN,Work-Related,"Grady HS to Home, via BL w/ Rebecca Serna!",30080.838760,9.254977,2016-08-23 19:53:33
2734,34242,1733,1733.0,2.230321e+06,1.373114e+06,2.233756e+06,1.361323e+06,0.0,-1.0,-1.0,0.0,True,NaN,NaN,Social,NaN,18289.733133,9.167505,2016-08-28 14:11:08
2735,34289,1733,1733.0,2.245986e+06,1.369955e+06,2.257314e+06,1.373259e+06,1.0,-1.0,-1.0,1.0,True,NaN,NaN,Social,NaN,14594.897768,10.451568,2016-08-30 19:35:19
2736,34290,1733,1733.0,2.257029e+06,1.371364e+06,2.229939e+06,1.374097e+06,-1.0,0.0,-1.0,0.0,False,NaN,NaN,Social,NaN,35551.538594,12.431327,2016-08-30 21:37:29


In [9]:
trips_df.columns

Index(['tripid', 'userid', 'remapped_userid', 'start_X', 'start_Y', 'end_X',
       'end_Y', 'start_label', 'end_label', 'sort_start_label',
       'sort_end_label', 'reverse_trajectory', 'trip_patterns',
       'trip_pattern_prevalence', 'trip_type', 'description',
       'total_distance_ft', 'avg_speed_mph'],
      dtype='object')